<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/09_Transformers/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usando RAG (Retrieval-Augmented Generation), vamos a armar un sistema capaz de responder preguntas _basadas en evidencia_, es decir, basadas en una fuente de información específica.

## Configuración del entorno

In [1]:
!pip install -qU google-generativeai langchain-google-genai langchain-google-vertexai\
    langchain-community langchainhub faiss-cpu sentence-transformers datasets watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 823.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/

In [2]:
%reload_ext watermark

In [3]:
%watermark -vmp \
google.generativeai,langchain_google_genai,langchain_google_vertexai,langchain_community,langchainhub,faiss,datasets,transformers,sentence_transformers,torch,pandas,numpy

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

google.generativeai      : 0.8.3
langchain_google_genai   : 2.0.1
langchain_google_vertexai: 2.0.5
langchain_community      : 0.3.3
langchainhub             : 0.1.21
faiss                    : 1.9.0
datasets                 : 3.0.2
transformers             : 4.44.2
sentence_transformers    : 3.2.1
torch                    : 2.5.0+cu121
pandas                   : 2.2.2
numpy                    : 1.26.4

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



Para usar la API de Gemini, hay que obtener una API key desde [acá](https://makersuite.google.com/app/apikey).

Luego, en Colab, añadir la clave en "Secrets" (🔑 en el panel izquierdo). Darle el nombre `GOOGLE_API_KEY`.

In [5]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU".

En este caso nos va a servir para hacer inferencia con un encoder para obtener la representación vectorial de cada documento del corpus.

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Dataset

Vamos a intentar responder preguntas sobre recetas del dataset _recetas de la abuela_.

El objetivo va a ser intentar responder preguntas del estilo "¿Qué puedo cocinar para la cena con tomates y sin cebolla?", tal que las respuestas estén basadas en el dataset.

Columnas del dataset:

* Id: Identificador numérico.
* Nombre: Nombre de la receta.
* URL: Origen web.
* Ingredientes: Alimentos usados.
* Pasos: Pasos de preparación.
* País: Código ISO_A3/país originario de la receta.
* Duracion (HH:MM): Tiempo estimado de preparación.
* Categoria: Tipo de receta (ej. vegetarianos, pastas, salsas, postres, cerdo, pollo etc).
* Contexto: Entorno de uso/consumo o contexto de la receta.
* Valoracion y Votos: Valoración 1-5 y número de votos.
* Comensales: Número de raciones.
* Tiempo: Tiempo del plato (ej: Desayuno, entrante, principal, acompañamiento, etc.)
* Dificultad: Grado de dificultad (alto/medio/bajo)
* Valor nutricional: Características básicas: 1) Nivel calorías/sodio (alto/medio/bajo), 2) Ausencia de grasas/grasas trans/colesterol/azúcar y 3) Nivel de fibra.

In [7]:
import datasets

dataset = datasets.load_dataset("somosnlp/RecetasDeLaAbuela", "version_1")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

main.csv:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [8]:
dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DatasetDict({
    train: Dataset({
        features: ['Id', 'Nombre', 'URL', 'Ingredientes', 'Pasos', 'Pais', 'Duracion', 'Categoria', 'Contexto', 'Valoracion y Votos', 'Comensales', 'Tiempo', 'Dificultad', 'Valor nutricional'],
        num_rows: 20236
    })
})

Vamos a representar cada receta (documento) como la concatenación de id, nombre, categoria, ingredientes, duracion, valor nutricional.

El objetivo es que el agente pueda responder acerca de los ingredientes que llevan las recetas; el pipeline podría concluir con un sistema que devuelva los pasos de las recetas relevantes, si las hubiera (y para esto no hace falta RAG).

In [9]:
import pandas as pd

df = dataset["train"].to_pandas()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Id,Nombre,URL,Ingredientes,Pasos,Pais,Duracion,Categoria,Contexto,Valoracion y Votos,Comensales,Tiempo,Dificultad,Valor nutricional
0,1,Tacos Dorados de Papa,https://www.mexicoenmicocina.com/tacos-dorados...,"½ taza de cilantro finamente picado, 1 taza de...",Pon las papas enteras en una olla mediana y cú...,MEX,01:00,vegetarianos,None,None,6,None,None,"Alto en calorías, Alto en grasas, Alto en sodio"
1,2,Ensalada de Chayotes,https://www.mexicoenmicocina.com/receta-ensala...,"½ cucharadita sal, ¼ cucharadita de pimienta r...",Coloca los chayotes en una cacerola y cúbralos...,MEX,00:25,vegetarianos,None,None,4,None,None,"Bajo en calorías, Sin grasa, Alto en fibra"


In [11]:
data_template = """
Receta: {nombre}
Categoría: {categoria}
Ingredientes: {ingredientes}
Duración: {duracion}
Valor nutricional: {valor_nutricional}
""".strip()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Si duracion es None, reemplazar por "indeterminado"
duracion = df["Duracion"].fillna("indeterminada")
categoria = df["Categoria"].fillna("indeterminada")
# Si ingredientes se parece a una lista, convertimos a joined string:
mask = df["Ingredientes"].apply(lambda x: x.startswith("["))
ingred_lists = df["Ingredientes"][mask].apply(eval)
ingred_strs = ingred_lists.apply(", ".join)
ingredientes = df["Ingredientes"].copy()
ingredientes[mask] = ingred_strs

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
ingredientes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ingredientes
0,"½ taza de cilantro finamente picado, 1 taza de..."
1,"½ cucharadita sal, ¼ cucharadita de pimienta r..."
2,"½ Chile Serrano cortado en pequeños trocitos*,..."
3,"½ cucharadita Sal, 2 cucharadas de aceite vege..."
4,"⅓ taza de cebolla blanca en cubitos, Sal al gu..."
...,...
20231,"1/4 kg frejol blanco, 1 diente ajo triturado..."
20232,"Zapallo, 4 papas, Al gusto sal, 2 zanahori..."
20233,"2 pechugas de pollo, 1 vaso avena, 1 ceboll..."
20234,"240 g chocolate negro sin azÃºcar, 40 gr cre..."


In [14]:
docs = []

for i, row in df.iterrows():
    doc = data_template.format(
        nombre=row["Nombre"],
        categoria=categoria.values[i],
        ingredientes=ingredientes.values[i],
        duracion=duracion.values[i],
        valor_nutricional=row["Valor nutricional"],
    )
    docs.append(doc)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# Cantidad de palabras por doc (proxy de tokens)
pd.Series(docs).str.split().apply(len).describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
count,20236.000000
mean,62.557818
std,20.190312
min,18.000000
25%,48.000000
50%,60.000000
75%,73.000000
max,232.000000


## "Encoding" o "vectorización" de los documentos

Vamos a guardar cada documento en un **vector store**. Esto es una base de datos especializada en guardar documentos representados como vectores densos (**embeddings**).

Necesitamos tomar dos decisiones:

1. El modelo con el que vamos a obtener los vectores, o **retriever**. Acá vamos a usar el modelo de Microsoft `multilingual-e5-small`.
2. La **base de datos** para los embeddings. Hay muchas opciones, algunas de las populares son Chroma, Pinecone, FAISS. Acá vamos a usar FAISS.

Para integrar el vector store con el LLM vamos a usar **LangChain**, que es un framework diseñado para simplificar la creación de aplicaciones basadas en LLMs.

En este caso vamos a extraer embeddings directamente a partir de los documentos, pero muchas veces se estila "partir" a los documentos en unidades más pequeñas antes de vectorizarlos. Para hacer esto podemos usar los [TextSplitters de LangChain](https://python.langchain.com/v0.2/docs/how_to/#text-splitters).

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': device}
encode_kwargs = {
    'normalize_embeddings': True,
    'prompt': "passage: ", # En E5 los documentos deben llevar el prefijo "passage:"
    "batch_size": 32,
    "device": device,
    }
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# por default usa mean pooling

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-16-fbd278b39047>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [17]:
vectors_example = embedding_model.embed_documents(docs[:2])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Longitud del embedding de cada documento:
print([len(vec) for vec in vectors_example])

[384, 384]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
print(type(vectors_example[0]))
print(vectors_example[0][:10])

<class 'list'>
[0.005876157432794571, -0.052538760006427765, -0.028926948085427284, -0.08407676964998245, 0.06541994214057922, -0.05548569932579994, 0.030902404338121414, 0.029995037242770195, 0.05146561563014984, -0.0002634357661008835]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
%%time
from langchain_community.vectorstores import FAISS

metadatas = df[["Id", "URL"]].to_dict("records") # podemos guardar metadara asociada a cada documento
vector_store = FAISS.from_texts(docs, embedding_model, metadatas=metadatas)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 51.6 s, sys: 404 ms, total: 52 s
Wall time: 49.8 s


In [21]:
# En E5 la query debe llevar el prefijo "query:"
query_vec = embedding_model.embed_query("query: una receta con frutillas y duraznos")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
len(query_vec)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


384

In [23]:
retriever = vector_store.as_retriever(
    search_type="similarity", search_kwargs={"k": 6})

# Algnos search_kwargs útiles son:
# * score_threshold: Minimum relevance threshold for similarity_score_threshold
# * filter: Filter by document metadata

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
retrieved_docs = retriever.invoke("query: pescado con ingredientes agridulces")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
for doc in retrieved_docs:
    print(doc.page_content)
    print()

Receta: montadito de bacalao con ajoaceite
Categoría: indeterminada
Ingredientes: pan, bacalao, ajoaceite, cebollino
Duración: indeterminada
Valor nutricional: Bajo en calorías, Sin grasa, Alto en grasas

Receta: estofado de pescado con verduras
Categoría: indeterminada
Ingredientes: 1 cebolla, 2 tomates, aceite, ajo, agua, corvina, verduras
Duración: indeterminada
Valor nutricional: Bajo en calorías, Sin grasa, Alto en grasas

Receta: cinta con brocolis
Categoría: indeterminada
Ingredientes: Brócoli, Ajo, Mantequilla, Queso parmesano, Pasta (cintas)
Duración: indeterminada
Valor nutricional: Alto en calorías, Alto en grasas, Alto en sodio

Receta: parrillada de pescado
Categoría: indeterminada
Ingredientes: 1 kilo de pescados variados, 2 dl. aceite, zumo de un limon, oregano, sal, pimienta, 3 limones
Duración: indeterminada
Valor nutricional: Alto en grasas, Alto en calorías, Sin sodio

Receta: maqluba de berenjenas
Categoría: indeterminada
Ingredientes: 1 kg de berenjenas, 1/2 kg de 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
print(retrieved_docs[0].metadata)

{'Id': 2704, 'URL': 'https://www.recetasgratis.net/receta-de-montadito-de-bacalao-con-ajoaceite-34658.html'}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## RAG

Vamos a integrar el retriever en el sistema de RAG.

El [hub de LangChain](https://smith.langchain.com/hub) viene con muchos prompts que podemos usar directamente o como inspiración.

In [27]:
from langchain import hub

langchain_prompt = hub.pull("rlm/rag-prompt")
langchain_prompt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
res = llm.invoke("Hola, qué tal, cómo estas?")
print(res)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


content='¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás? 😊 \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-73080430-7943-4afa-859a-c82f56664dd1-0' usage_metadata={'input_tokens': 9, 'output_tokens': 18, 'total_tokens': 27}


In [30]:
from langchain_core.prompts import PromptTemplate

template = (
    """Usa los siguientes documentos para responder la consulta del usuario al final."""
    """Si no sabe la respuesta, simplemente diga que no la sabe, no intente inventar una respuesta."""
    """Si tiene una respuesta, incluya el o los IDs y URLs pertinentes en la respuesta.\n\n"""
    """{context}\n\n"""
    """Pregunta: {input}\n\n"""
    """Respuesta:"""
)

def format_docs(docs):
    """Aplicar a cada documento el formato:
    '[ID=...] content (URL=...)'
    """
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"[ID={doc.metadata['Id']}] {doc.page_content} (URL={doc.metadata['URL']})")
    return "\n\n".join(formatted_docs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# por ejemplo:
print(format_docs(retrieved_docs))

[ID=2704] Receta: montadito de bacalao con ajoaceite
Categoría: indeterminada
Ingredientes: pan, bacalao, ajoaceite, cebollino
Duración: indeterminada
Valor nutricional: Bajo en calorías, Sin grasa, Alto en grasas (URL=https://www.recetasgratis.net/receta-de-montadito-de-bacalao-con-ajoaceite-34658.html)

[ID=11833] Receta: estofado de pescado con verduras
Categoría: indeterminada
Ingredientes: 1 cebolla, 2 tomates, aceite, ajo, agua, corvina, verduras
Duración: indeterminada
Valor nutricional: Bajo en calorías, Sin grasa, Alto en grasas (URL=https://www.recetasgratis.net/receta-de-estofado-de-pescado-con-verduras-52715.html)

[ID=19132] Receta: cinta con brocolis
Categoría: indeterminada
Ingredientes: Brócoli, Ajo, Mantequilla, Queso parmesano, Pasta (cintas)
Duración: indeterminada
Valor nutricional: Alto en calorías, Alto en grasas, Alto en sodio (URL=https://www.recetasgratis.net/receta-de-cinta-con-brocolis-18190.html)

[ID=12427] Receta: parrillada de pescado
Categoría: indetermi

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_prompt = PromptTemplate.from_template(template)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | rag_prompt
    | llm
    | StrOutputParser()
)

retrieve_docs = (lambda x: f'query: {x["input"]}') | retriever

chain = (
    RunnablePassthrough
        .assign(context=retrieve_docs)
        .assign(answer=rag_chain_from_docs)
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cada uno de estos componentes (retriever, prompt, llm, etc.) son instancias de Runnable. Implementan los mismos métodos-- como sync y async .invoke, .stream, o .batch, y pueden conectarse a un RunnableSequence - otro Runnable - mediante el operador |.

Fuentes:

* https://python.langchain.com/v0.2/docs/how_to/qa_sources/#custom-lcel-implementation
* https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel


In [33]:
response = chain.invoke({"input": "¿Qué puedo hacer con pescado?"})

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
response

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'input': '¿Qué puedo hacer con pescado?',
 'context': [Document(metadata={'Id': 11833, 'URL': 'https://www.recetasgratis.net/receta-de-estofado-de-pescado-con-verduras-52715.html'}, page_content='Receta: estofado de pescado con verduras\nCategoría: indeterminada\nIngredientes: 1 cebolla, 2 tomates, aceite, ajo, agua, corvina, verduras\nDuración: indeterminada\nValor nutricional: Bajo en calorías, Sin grasa, Alto en grasas'),
  Document(metadata={'Id': 12427, 'URL': 'https://www.recetasgratis.net/receta-de-parrillada-de-pescado-15056.html'}, page_content='Receta: parrillada de pescado\nCategoría: indeterminada\nIngredientes: 1 kilo de pescados variados, 2 dl. aceite, zumo de un limon, oregano, sal, pimienta, 3 limones\nDuración: indeterminada\nValor nutricional: Alto en grasas, Alto en calorías, Sin sodio'),
  Document(metadata={'Id': 10996, 'URL': 'https://www.recetasgratis.net/receta-de-pizza-de-pescado-35127.html'}, page_content='Receta: pizza de pescado\nCategoría: indeterminada\nI

In [35]:
print(response["answer"])

Puedes hacer varias cosas con pescado, como:

* **Estofado de pescado con verduras:** [ID=11833] (https://www.recetasgratis.net/receta-de-estofado-de-pescado-con-verduras-52715.html)
* **Parrillada de pescado:** [ID=12427] (https://www.recetasgratis.net/receta-de-parrillada-de-pescado-15056.html)
* **Pizza de pescado:** [ID=10996] (https://www.recetasgratis.net/receta-de-pizza-de-pescado-35127.html)
* **Pescado a la gallega caldeirada:** [ID=12237] (https://www.recetasgratis.net/receta-de-pescado-a-la-gallega-caldeirada-26124.html)
* **Tiradito de pescado:** [ID=11803] (https://www.recetasgratis.net/receta-de-tiradito-de-pescado-75305.html)
* **Tacos de pescado empanizado:** [ID=2708] (https://www.recetasgratis.net/receta-de-tacos-de-pescado-empanizado-34536.html) 



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
response = chain.invoke({"input": "¿Qué puedo hacer con cebolla y tomate?"})

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
response

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'input': '¿Qué puedo hacer con cebolla y tomate?',
 'context': [Document(metadata={'Id': 18599, 'URL': 'https://www.recetasgratis.net/receta-de-guarnicion-de-cebollas-y-tomates-al-horno-con-huevo-51558.html'}, page_content='Receta: guarnicion de cebollas y tomates al horno con huevo\nCategoría: indeterminada\nIngredientes: 2 tomates, 2 cebollas blancas, 1 huevo, sal, 100 gr miga de pan\nDuración: indeterminada\nValor nutricional: Alto en calorías, Alto en grasas, Sin sodio o sin sal'),
  Document(metadata={'Id': 3036, 'URL': 'https://www.recetasgratis.net/receta-de-seviche-de-tomate-con-cebolla-16106.html'}, page_content='Receta: seviche de tomate con cebolla\nCategoría: indeterminada\nIngredientes: 6 Tomates maduros, 1 cebolla cabezona roja, 1 cebolla cabezona blanca, 2 cds de pasta de tomate, 2 cds de salsa de tomate, 1 chorrito de vinagre, Sal, Pimienta, Azucar\nDuración: indeterminada\nValor nutricional: Bajo en calorías, Sin grasa, Sin grasas trans'),
  Document(metadata={'Id': 3

In [38]:
import textwrap

from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *').replace("\n", "\n\n")
    # predicate: hack para indentar todas las líneas
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
to_markdown(response["answer"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


> Puedes hacer muchas cosas con cebolla y tomate. Aquí hay algunas ideas basadas en los documentos:
> 
> 
> 
> * **Guarnición de cebollas y tomates al horno con huevo:** [ID=18599] (https://www.recetasgratis.net/receta-de-guarnicion-de-cebollas-y-tomates-al-horno-con-huevo-51558.html)
> 
> * **Seviche de tomate con cebolla:** [ID=3036] (https://www.recetasgratis.net/receta-de-seviche-de-tomate-con-cebolla-16106.html)
> 
> * **Arroz con cebolla, morrón y tomate:** [ID=3724] (https://www.recetasgratis.net/receta-de-arroz-con-cebolla-morron-y-tomate-32171.html)
> 
> * **Estofado de pescado con verduras:** [ID=11833] (https://www.recetasgratis.net/receta-de-estofado-de-pescado-con-verduras-52715.html) 
> 


In [40]:
for doc in response["context"]:
    print(doc.metadata)
    print(doc.page_content)
    print()

{'Id': 18599, 'URL': 'https://www.recetasgratis.net/receta-de-guarnicion-de-cebollas-y-tomates-al-horno-con-huevo-51558.html'}
Receta: guarnicion de cebollas y tomates al horno con huevo
Categoría: indeterminada
Ingredientes: 2 tomates, 2 cebollas blancas, 1 huevo, sal, 100 gr miga de pan
Duración: indeterminada
Valor nutricional: Alto en calorías, Alto en grasas, Sin sodio o sin sal

{'Id': 3036, 'URL': 'https://www.recetasgratis.net/receta-de-seviche-de-tomate-con-cebolla-16106.html'}
Receta: seviche de tomate con cebolla
Categoría: indeterminada
Ingredientes: 6 Tomates maduros, 1 cebolla cabezona roja, 1 cebolla cabezona blanca, 2 cds de pasta de tomate, 2 cds de salsa de tomate, 1 chorrito de vinagre, Sal, Pimienta, Azucar
Duración: indeterminada
Valor nutricional: Bajo en calorías, Sin grasa, Sin grasas trans

{'Id': 3724, 'URL': 'https://www.recetasgratis.net/receta-de-arroz-con-cebolla-morron-y-tomate-32171.html'}
Receta: arroz con cebolla morron y tomate
Categoría: indeterminad

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


El pipeline podría seguir con un sistema de búsqueda más sencillo que devuelve los pasos de la receta según los IDs.

In [41]:
response = chain.invoke({"input": "¿Qué puedo hacer con mayonesa y dulce de leche?"})
to_markdown(response["answer"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


> No lo sé. Los documentos no contienen información sobre qué se puede hacer con mayonesa y dulce de leche. 
> 


In [42]:
for doc in response["context"]:
    print(doc.metadata)
    print(doc.page_content)
    print()

{'Id': 16546, 'URL': 'https://www.recetasgratis.net/receta-de-mayonesa-de-leche-con-thermomix-30806.html'}
Receta: mayonesa de leche con thermomix
Categoría: []
Ingredientes: 200 mililitros de Leche, 2 dientes de Ajo, 400 gramos de Aceite, 1 pizca de Sal
Duración: 00:15
Valor nutricional: Alto en calorías, Alto en grasas, Alto en sodio

{'Id': 16608, 'URL': 'https://www.recetasgratis.net/receta-de-mayonesa-de-leche-70691.html'}
Receta: mayonesa de leche
Categoría: []
Ingredientes: 150 mililitros de leche, 1 diente de ajo, 1 cucharada postre de sal, 1 cucharadita de orégano, 3 ramas de perejil, Aceite
Duración: 00:15
Valor nutricional: Alto en calorías, Alto en grasas, Alto en sodio

{'Id': 15977, 'URL': 'https://www.recetasgratis.net/receta-de-cuajada-con-thermomix-30422.html'}
Receta: cuajada con thermomix
Categoría: []
Ingredientes: 1 litro de Leche entera, 2 sobres de Cuajada Royal
Duración: 00:15
Valor nutricional: Alto en calorías, Alto en grasas, Alto en sodio

{'Id': 16929, 'URL

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
response = chain.invoke({"input": "¿Qué puedo hacer con dulce de leche?"})
to_markdown(response["answer"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
response = chain.invoke({"input": "¿Cuánto tiempo lleva hacer una mayonesa?"})
to_markdown(response["answer"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


> La mayonesa puede tardar entre 10 y 30 minutos en prepararse. 
> 
> 
> 
> * [ID=16929] Receta: mayonesa - 30 minutos (URL=https://www.recetasgratis.net/receta-de-mayonesa-26931.html)
> 
> * [ID=16690] Receta: mayonesa facil - 15 minutos (URL=https://www.recetasgratis.net/receta-de-mayonesa-facil-57527.html)
> 
> * [ID=16643] Receta: mayonesa casera - 10 minutos (URL=https://www.recetasgratis.net/receta-de-mayonesa-casera-62554.html) 
> 


## Otros

* Agente inicial que convierta el prompt del usuario en una query útil: https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/
* Combinar resultados de muchos retrievers: https://python.langchain.com/v0.2/docs/how_to/ensemble_retriever/
* Obtener documentos completos a partir de fragmentos: https://python.langchain.com/v0.2/docs/how_to/parent_document_retriever/
* Búsqueda híbrida: https://python.langchain.com/v0.2/docs/how_to/hybrid/
* Function/tool calling: https://python.langchain.com/v0.2/docs/how_to/tool_calling/



